### Import Libraries

In [ ]:
# import required libraries
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import os
import re
from pathlib import Path

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op


### YAML Parameters

In [ ]:
#tag cell with parameters
PROJECT_ID =  ''
BUCKET_NAME=''
DATASET_ID = ''
RESOURCE_BUCKET = ''
FILE_BUCKET = ''
REGION = ''
MODEL_ID = '5070'

In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
DATASET_ID = 'telus_rewards'
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
MODEL_ID = '5070'

### Service Parameters

In [ ]:
SERVICE_TYPE = 'telus_rewards'
SERVICE_TYPE_NAME = 'telus-rewards'
TABLE_ID = 'telus_rwrd_redemption_targets'
REGION = 'northamerica-northeast1'

### Pipeline Parameters

In [ ]:
STACK_NAME = 'telus_rewards'
TRAIN_PIPELINE_NAME_PATH = 'telus_rewards/training_pipeline'
PREDICT_PIPELINE_NAME_PATH = 'telus_rewards/predicting_pipeline'
TRAIN_PIPELINE_NAME = 'telus-rewards-train-pipeline' # Same name as pulumi.yaml
PREDICT_PIPELINE_NAME = 'telus-rewards-predict-pipeline' # Same name as pulumi.yaml
TRAIN_PIPELINE_DESCRIPTION = 'telus-rewards-train-pipeline'
PREDICT_PIPELINE_DESCRIPTION = 'telus-rewards-predict-pipeline'
PIPELINE_ROOT = f"gs://{BUCKET_NAME}"
REGION = "northamerica-northeast1"

In [ ]:
TRAINING_DATASET_TABLE_NAME = 'bq_telus_rewards_pipeline_dataset'
TRAINING_DATASET_SP_NAME = 'bq_sp_telus_rewards_pipeline_dataset'

### Import Pipeline Components

In [ ]:
# download required component files to local
prefix = f'{STACK_NAME}/{TRAIN_PIPELINE_NAME_PATH}/components/'
dl_dir = 'components/'

storage_client = storage.Client()
bucket = storage_client.bucket(RESOURCE_BUCKET)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs: # download each file that starts with "prefix" into "dl_dir"
    if blob.name.endswith("/"):
        continue
    file_split = blob.name.split(prefix)
    file_path = f"{dl_dir}{file_split[-1]}"
    directory = "/".join(file_path.split("/")[0:-1])
    Path(directory).mkdir(parents=True, exist_ok=True)
    blob.download_to_filename(file_path) 

# import main pipeline components
from components.bq_create_dataset import bq_create_dataset
from components.preprocess import preprocess
from components.train_and_save_model import train_and_save_model


### Date Parameters

In [ ]:
scoringDate = date(2022, 7, 1)  # date.today() - relativedelta(days=2)- relativedelta(months=30)
valScoringDate = date(2023, 1, 1)  # scoringDate - relativedelta(days=2)

# training dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START = (scoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END = (scoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

# validation dates
SCORE_DATE_VAL = valScoringDate.strftime('%Y%m%d')
SCORE_DATE_VAL_DASH = valScoringDate.strftime('%Y-%m-%d')
SCORE_DATE_VAL_MINUS_6_MOS_DASH = ((valScoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_START_DASH = (valScoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_END_DASH = ((valScoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START_VAL = (valScoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END_VAL = (valScoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried


### Pipeline

In [ ]:
# library imports
from kfp.v2 import compiler
from google.cloud.aiplatform import pipeline_jobs
@dsl.pipeline(
    name=TRAIN_PIPELINE_NAME, 
    description=TRAIN_PIPELINE_DESCRIPTION
    )
def pipeline(
        project_id: str = PROJECT_ID,
        region: str = REGION,
        resource_bucket: str = RESOURCE_BUCKET, 
        file_bucket: str = FILE_BUCKET
    ):
    
    import google.oauth2.credentials
    token = !gcloud auth print-access-token
    token_str = token[0]
    
    # ----- create training set --------
    bq_create_training_dataset_op = bq_create_dataset(score_date=SCORE_DATE_DASH,
                          score_date_delta=SCORE_DATE_DELTA,
                          project_id=PROJECT_ID,
                          dataset_id=DATASET_ID,
                          region=REGION,
                          promo_expiry_start=PROMO_EXPIRY_START, 
                          promo_expiry_end=PROMO_EXPIRY_END, 
                          v_start_date=SCORE_DATE_MINUS_6_MOS_DASH,
                          v_end_date=SCORE_DATE_LAST_MONTH_END_DASH, 
                          token = token_str)
    
    bq_create_training_dataset_op.set_memory_limit('128G')
    bq_create_training_dataset_op.set_cpu_limit('16')
    
    # ----- preprocessing train data --------
    preprocess_train_op = preprocess(
        pipeline_dataset=TRAINING_DATASET_TABLE_NAME, 
        save_data_path='gs://{}/{}_train.csv.gz'.format(FILE_BUCKET, SERVICE_TYPE),
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID, 
        token = token_str
    )

    preprocess_train_op.set_memory_limit('128G')
    preprocess_train_op.set_cpu_limit('16')
    
    # ----- create validation set --------
    bq_create_validation_dataset_op = bq_create_dataset(score_date=SCORE_DATE_VAL_DASH,
                          score_date_delta=SCORE_DATE_VAL_DELTA,
                          project_id=PROJECT_ID,
                          dataset_id=DATASET_ID,
                          region=REGION,
                          promo_expiry_start=PROMO_EXPIRY_START_VAL, 
                          promo_expiry_end=PROMO_EXPIRY_END_VAL, 
                          v_start_date=SCORE_DATE_VAL_MINUS_6_MOS_DASH,
                          v_end_date=SCORE_DATE_VAL_LAST_MONTH_END_DASH, 
                          token = token_str)
    

    bq_create_validation_dataset_op.set_memory_limit('128G')
    bq_create_validation_dataset_op.set_cpu_limit('16')

    
    # ----- preprocessing validation data --------
    preprocess_validation_op = preprocess(
        pipeline_dataset=TRAINING_DATASET_TABLE_NAME, 
        save_data_path='gs://{}/{}_validation.csv.gz'.format(FILE_BUCKET, SERVICE_TYPE),
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID, 
        token = token_str
    )

    preprocess_validation_op.set_memory_limit('128G')
    preprocess_validation_op.set_cpu_limit('16')

    train_and_save_model_op = train_and_save_model(file_bucket=FILE_BUCKET,
                                                   service_type=SERVICE_TYPE,
                                                   score_date_dash=SCORE_DATE_DASH,
                                                   score_date_val_dash=SCORE_DATE_VAL_DASH,
                                                   project_id=PROJECT_ID,
                                                   dataset_id=DATASET_ID,
                                                   )

    train_and_save_model_op.set_memory_limit('128G')
    train_and_save_model_op.set_cpu_limit('16')

    preprocess_train_op.after(bq_create_training_dataset_op)
    bq_create_validation_dataset_op.after(preprocess_train_op)
    preprocess_validation_op.after(bq_create_validation_dataset_op)
    train_and_save_model_op.after(preprocess_validation_op)
    

### Reduced Pipeline - MODEL TRAINING ONLY

In [ ]:
# # library imports
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs
# @dsl.pipeline(
#     name=TRAIN_PIPELINE_NAME, 
#     description=TRAIN_PIPELINE_DESCRIPTION
#     )
# def pipeline(
#         project_id: str = PROJECT_ID,
#         region: str = REGION,
#         resource_bucket: str = RESOURCE_BUCKET, 
#         file_bucket: str = FILE_BUCKET
#     ):
    
#     import google.oauth2.credentials
#     token = !gcloud auth print-access-token
#     token_str = token[0]
    
#     train_and_save_model_op = train_and_save_model(file_bucket=FILE_BUCKET,
#                                                    service_type=SERVICE_TYPE,
#                                                    score_date_dash=SCORE_DATE_DASH,
#                                                    score_date_val_dash=SCORE_DATE_VAL_DASH,
#                                                    project_id=PROJECT_ID,
#                                                    dataset_id=DATASET_ID,
#                                                    )

#     train_and_save_model_op.set_memory_limit('128G')
#     train_and_save_model_op.set_cpu_limit('16')

#     train_and_save_model_op
    

### Run the Pipeline Job

In [ ]:
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs
# import json

# compiler.Compiler().compile(
#    pipeline_func=pipeline, package_path="pipeline.json"
# )

# # job = pipeline_jobs.PipelineJob(
# #                                    display_name=TRAIN_PIPELINE_NAME,
# #                                    template_path="pipeline.json",
# #                                    location=REGION,
# #                                    enable_caching=False,
# #                                    pipeline_root = PIPELINE_ROOT
# #                                 )

# token = os.popen('gcloud auth print-access-token').read()
# token = re.sub(f'\n$', '', token)
# credentials = google.oauth2.credentials.Credentials(token)

# job = pipeline_jobs.PipelineJob(
#     display_name=TRAIN_PIPELINE_NAME,
#     template_path="pipeline.json",
#     pipeline_root=PIPELINE_ROOT,
#     credentials=credentials,
#     project='divg-josh-pr-d1cc3a',
#     location=REGION,
#     enable_caching=True, 
# )

# job.run()

# # job.run(service_account = "notebook-service-account@divg-josh-pr-d1cc3a.iam.gserviceaccount.com")
# # job.run(service_account = f"bilayer-sa@{PROJECT_ID}.iam.gserviceaccount.com")


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name=TRAIN_PIPELINE_NAME,
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = PIPELINE_ROOT,
   location=REGION,
   enable_caching=False # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()